---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [1]:
sigma = 50                                                         # volatility in basis points
prds_per_year = 2
num_bonds = 6                                                      # number of bonds
maturities   = [1, 2, 3, 4, 5, 6]                                  # bond maturities
coupon_rates = [0.02, 0.02, 0.02, 0.02, 0.02, 0.02]                # coupon rates (annual)
yields       = [0.0175, 0.02, 0.0225, 0.024, 0.025, 0.026]         # yields (annual)
zero_maturity = 5                                                  # for zero-coupon plot

if not all(len(lst)==num_bonds for lst in [maturities, coupon_rates, yields]):
    print('Lists need to be of the same length!')

# FUNCTIONS FOR BOND PRICES AND FORWARD RATES

Unlike the "spot and forward rates" notebook, here we work with a general number of periods per year rather than six-month periods.

In [2]:
import numpy as np
from scipy.optimize import minimize

# calculate bond price for given yield to maturity or for given spot rates
def price(**kwargs):
    c = 100 * kwargs['coupon'] / 2
    maturity = kwargs['maturity']
    if 'yld' in kwargs.keys():
        yld = kwargs['yld']
        n = int(2 * maturity)
        return c * np.sum((1 + yld/2) ** np.arange(-1, -n - 1, -1)) + 100 / (1 + yld/2) ** n
    else:
        n = int(prds_per_year * maturity)
        pphy = int(prds_per_year/2)  # periods per half year
        spots = np.array(kwargs['spots'])
        pv_factors = (1 + spots / prds_per_year) ** (-np.arange(1, len(spots) + 1))
        coupons = np.zeros(n)
        coupons[(pphy - 1)::pphy] = c
        return np.sum(coupons*pv_factors[:len(coupons)]) + 100*pv_factors[n-1]
    
# forward rates implied by spot rates
def forward_rates(spots):
    pphy = int(prds_per_year / 2)  # periods per half year
    future_factors = (1 + spots / 2) ** (np.arange(1, len(spots) + 1) / pphy)
    diff_logs = np.diff(np.log(future_factors))
    f = (np.exp(diff_logs)-1) * prds_per_year
    return np.concatenate(([spots[0]], f))

# FUNCTION TO COMPUTE SPOT RATES

We choose spot rates to match bond prices as closely as possible, except that we add a penalty for variation in implied forward rates, so the resulting forward rate curve is reasonably smooth.  

In [3]:
# difference between bond price and price implied by spot rates
def error(bond, spots):
    price1 = price(**bond)
    price2 = price(maturity=bond["maturity"], coupon=bond['coupon'], spots=spots)
    return np.log(price2 / price1)

# sum of squared errors between bond prices and prices implied by spot rates
# plus a penalty for variation in forward rates
def objective(bonds, spots):
    errors = [error(bond, spots) for bond in bonds]
    sse = np.sum(np.array(errors)**2)
    forwards = forward_rates(spots)
    diffs = np.sum(np.diff(np.log(1+forwards/prds_per_year))**2)
    return sse + 0.5*diffs

# choose spot rates to optimize the objective
def spot_rates(bonds):
    maturities = [bond['maturity'] for bond in bonds]
    n = int(np.max(maturities) * prds_per_year)
    result = minimize(lambda x: objective(bonds, x), [0.05]*n)
    return result.x if result.success==True else np.nan

# SPOT AND FORWARD RATES

In [4]:
bonds = [
    dict(
        maturity= maturities[i],
        coupon  = coupon_rates[i],
        yld     = yields[i]
    )
    for i in range(num_bonds)
]

spots    = spot_rates(bonds)
forwards = forward_rates(spots)

# INTEREST RATE TREE

We find the drift of the interest rate tree so that it matches the forward rates calculated above.  See, for example, Back, A Course in Derivative Securities, 2006.

In [5]:
n = prds_per_year * np.max(maturities)
dt = 1 / prds_per_year
m = len(forwards)
s = sigma / 10000   # volatility as decimal
f = forwards[:n+1] if m>=n+1 else np.concatenate((forwards, [forwards[-1]]*(n+1-m)))
    
term1 = np.log(1+f*dt)
a = np.exp(s*dt**(3/2)*np.arange(n+1))
term2 = np.log(a + 1/a)
term3 = np.log(2*(1+f[0]*dt))
phis = (term1 + term2 - term3) / dt

delta = sigma*np.sqrt(dt) / 10000
rate_tree =  [
    [spots[0] + phi + delta * (i - 2 * j) for j in range(i + 1)] 
    for i, phi in enumerate(phis)
]

# ZERO-COUPON TREE

We use the interest rate tree to price a zero-coupon bond for illustration.

In [6]:
n = int(prds_per_year * zero_maturity)
rates = [np.array(x) for x in rate_tree]

# initialize the tree at $100 at the last date
x = 100 * np.ones(n + 1)
lst = [x]
i = n - 1

# back up in the tree
while len(x) > 1:
    x = 0.5 * (x[:-1] + x[1:]) / (1 + rates[i] / prds_per_year)
    lst.insert(0, x)
    i -= 1
zero_tree = [list(x) for x in lst]


# FIGURES

In [7]:
import plotly.graph_objects as go

figs = []
for kind in ["rate", "zero"]:
    tree = rate_tree if kind=="rate" else zero_tree
    string = "%{y:.2%}<extra></extra>" if kind=='rate' else "$%{y:,.2f}<extra></extra>"
    spliced = []
    for a, b in zip(tree[1:], tree[:-1]):
        x = []
        for i in range(len(a)):
            x.append(a[i])
            try:
                x.append(b[i])
            except:
                pass
        spliced.append(x)
    fig = go.Figure()
    for i in range(len(tree) - 1):
        x = [1, 0, 1]
        for j in range(i):
            x.append(0)
            x.append(1)
        x = np.array(x) + i
        y = spliced[i]
        trace = go.Scatter(
            x=x*dt,
            y=y,
            mode="lines+markers",
            hovertemplate=string,
            marker=dict(size=10, color="blue"),
            line_color="blue",
        )
        fig.add_trace(trace)
    fig.update_layout(
        xaxis_title="Time (years)",
        yaxis_title="Annualized Short Rate" if kind=="rate" else "Zero Coupon Bond Price",
        yaxis_tickprefix = None if kind=="rate" else "$", 
        yaxis_tickformat=".2%" if kind=="rate" else ".0f",
        yaxis_autorange=None if kind=="rate" else "reversed",
        template="plotly_white",
        showlegend=False
    )
    figs.append(fig)

# INTEREST RATE TREE

In [8]:
figs[0].show()

# ZERO_COUPON TREE

In [9]:
figs[1].show()